# Assignment 2 - Restaurant and Consumer datasets
---

## Importing the data

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', 10000)

In [2]:
accepts = pd.read_csv('data/chefmozaccepts.csv')
restaurant_cuisine = pd.read_csv('data/chefmozcuisine.csv')
restaurant_hours = pd.read_csv('data/chefmozhours4.csv', names=['placeID', 'hours', 'days'])
restaurant_parking = pd.read_csv('data/chefmozparking.csv')
geoplaces = pd.read_csv('data/geoplaces2.csv', encoding='latin1')
user_rating = pd.read_csv('data/rating_final.csv')
data_user_cuisine = pd.read_csv('data/usercuisine.csv')
user_payment = pd.read_csv('data/userpayment.csv')
user_profile = pd.read_csv('data/userprofile.csv')

---
## Cleaning the data

In this section, I will be using the methods below. The fisrt one replaces missing values by Numpy's NaN. The second one computes the ratio of missing values for every column of a DataFrame. The third one replaces all the strings by their lowercase representation, so as to group related names.

In [3]:
def normalize(df):
    df = df.applymap(lambda x:x.lower() if type(x) == str else x)
    df.replace(to_replace=['?', 'None', 'none'], value=np.nan, inplace=True)
    return df

In [4]:
def columns_na_ratio(df):
    return (df.isna().sum() / len(df.index)).round(3)

In [5]:
def drop_empty_rows(df):
    print(int(0.9*len(df.columns)))
    return df.dropna(axis='index', thresh=5)

### Geoplaces

First of all, let's clean the geographical dataset by dropping useless columns and checking if it contains any duplicate rows.

In [6]:
geoplaces.drop(columns=['the_geom_meter', 'fax', 'zip', 'url'], inplace=True)
geoplaces = normalize(geoplaces)
print(geoplaces.duplicated().sum(), "duplicated rows")

0 duplicated rows


This dataset has no duplicated rows, let's group all unavailable values and check if some columns aren't worth looking at.

In [7]:
columns_na_ratio(geoplaces)

placeID           0.000
latitude          0.000
longitude         0.000
name              0.000
address           0.208
city              0.138
state             0.138
country           0.215
alcohol           0.000
smoking_area      0.538
dress_code        0.000
accessibility     0.000
price             0.000
Rambience         0.000
franchise         0.000
area              0.000
other_services    0.915
dtype: float64

We can see that 'other services' contains more than 90% of unkwown values, so let's drop this column.

In [8]:
geoplaces.drop(columns='other_services', inplace=True)

### User profile

In [9]:
user_profile = normalize(user_profile)
print(user_profile['userID'].duplicated().sum(), "duplicated rows")
columns_na_ratio(user_profile)

0 duplicated rows


userID              0.000
latitude            0.000
longitude           0.000
smoker              0.022
drink_level         0.000
dress_preference    0.036
ambience            0.043
transport           0.051
marital_status      0.029
hijos               0.080
birth_year          0.000
interest            0.217
personality         0.000
religion            0.217
activity            0.051
color               0.000
weight              0.000
budget              0.051
height              0.000
dtype: float64

We made sure that each row represents a different user, as there are no duplicated userID.
We can drop 'interest' and 'religion' columns as they both have 20% unknown values and we won't be using them in the questions.

In [10]:
user_profile.drop(columns=['interest', 'religion'], inplace=True)

### User payments

In [11]:
user_payment = normalize(user_payment)
print(user_payment.duplicated().sum(), "duplicated rows")
print(user_payment['userID'].duplicated().sum(), "duplicated rows")
columns_na_ratio(user_payment)

0 duplicated rows
44 duplicated rows


userID      0.0
Upayment    0.0
dtype: float64

This DataFrame is already clean, with no missing value and no duplicated rows. It is important to note that some clients have multiple means of payment.

## User ratings

In [12]:
user_rating = normalize(user_rating)
print(user_rating.duplicated().sum(), "duplicated rows")
columns_na_ratio(user_rating)

0 duplicated rows


userID            0.0
placeID           0.0
rating            0.0
food_rating       0.0
service_rating    0.0
dtype: float64

This DataFrame is already clean.

## Restaurant Hours

In [13]:
restaurant_hours = normalize(restaurant_hours)
print(restaurant_hours.duplicated().sum(), "duplicated rows")
columns_na_ratio(restaurant_hours)

271 duplicated rows


placeID    0.0
hours      0.0
days       0.0
dtype: float64

This DataFrame has a lot of duplicated rows, let's remove them.

In [14]:
restaurant_hours.drop_duplicates(inplace=True)
print(restaurant_hours.duplicated().sum(), "duplicated rows")

0 duplicated rows


## Restaurant Cuisine

In [15]:
restaurant_cuisine = normalize(restaurant_cuisine)
print(restaurant_cuisine.duplicated().sum(), "duplicated rows")
print(restaurant_cuisine['placeID'].duplicated().sum(), "duplicated restaurants")
columns_na_ratio(restaurant_cuisine)

0 duplicated rows
147 duplicated restaurants


placeID     0.0
Rcuisine    0.0
dtype: float64

There are no duplicated rows and this DataFrame is already clean. It is important to note that some restaurants offer multiple types of cuisine.

## Restaurant Parking

In [16]:
restaurant_parking = normalize(restaurant_parking)
print(restaurant_parking.duplicated().sum(), "duplicated rows")
print(restaurant_parking['placeID'].duplicated().sum(), "duplicated rows")
columns_na_ratio(restaurant_parking)

0 duplicated rows
27 duplicated rows


placeID        0.000
parking_lot    0.496
dtype: float64

As we have observed before, this DataFrame does not contain duplicated rows but some restaurant offer multiple types of parking. We can also see that nearly half of the restaurants do not have a parking.

## Payments Accepted

In [17]:
accepts = normalize(accepts)
print(accepts.duplicated().sum(), "duplicated rows")
columns_na_ratio(accepts)

0 duplicated rows


placeID     0.0
Rpayment    0.0
dtype: float64

This DataFrame does not require cleaning.

---
## Questions - Part 1

**Question 1**
What are the names of different restaurants in the state of 'tamaulipas' ?

In [18]:
geoplaces[geoplaces['state'] == 'tamaulipas']['name']

3             little pizza emilio portes gil
4                              carnitas_mata
6                         taqueria el amigo 
8                   pollo_frito_buenos_aires
19                            tacos el guero
41                    hamburguesas la perica
73                                palomo tec
76                        tacos correcaminos
86             carreton de flautas y migadas
89                        gorditas dona tota
94                             little cesarz
103    carnitas mata  calle 16 de septiembre
106                       puesto de gorditas
109    carnitas mata calle emilio portes gil
122                                tacos abi
123                    la perica hamburguesa
Name: name, dtype: object

**Question 2**
How many different customers used public transport for going to the
restaurants ?

For this question, I assume each entry in 'user profile' is someone who actually went to the restaurants, as this dataset is collected from people going to restaurants.

In [19]:
user_profile['transport'].value_counts()

public       82
car owner    35
on foot      14
Name: transport, dtype: int64

We see that 82 customers used public transport.

**Question 3** What is the least popular payment method among customers ?


In [20]:
user_payment['Upayment'].value_counts()

cash                   131
bank_debit_cards        22
visa                    17
mastercard-eurocard      4
american_express         3
Name: Upayment, dtype: int64

In [21]:
user_payment[user_payment['Upayment'] == 'american_express'].count() / user_payment.count()

userID      0.016949
Upayment    0.016949
dtype: float64

The american express is the least popular payment method, with less than 2% of payments done via this method.

**Question 4** How many (different) restaurants work until 19:00 in the evenings ?

This question is ambiguous. Does it mean restaurants working *at least* until 19:00 or restaurants working until 19:00 and then closing ? In the answer, I assume the question seeks to find restaurants closing at 19:00.

In [22]:
restaurant_hours[restaurant_hours['hours'].str.contains('19:00;')]['placeID'].nunique()

28

28 different restaurants are open until 19:00, and then close.

**Question 5** Which type of cooking practice (rcuisine) is the most common among
restaurants ?

In [23]:
restaurant_cuisine['Rcuisine'].value_counts().head()

mexican          239
international     62
american          59
dutch-belgian     55
italian           42
Name: Rcuisine, dtype: int64

In [24]:
restaurant_cuisine[restaurant_cuisine['Rcuisine'] == 'mexican'].count() / restaurant_cuisine.count()

placeID     0.260917
Rcuisine    0.260917
dtype: float64

The most common cooking practice is mexican food, with more than 25% of the restaurants serving mexican.

**Question 6** What is the percentage of customers who were born between 1980 and
1990 ?

In [25]:
(user_profile[(user_profile['birth_year'] >= 1980) & (user_profile['birth_year'] <= 1990)].count() / user_profile.count())['userID']

0.6304347826086957

63% of customers were born between 1980 and 1990 included.

**Question 7** What is the percentage of students with a medium budget preferring
walking to the restaurants ?

In [26]:
students_medium = user_profile[(user_profile['activity'] == 'student') & (user_profile['budget'] == 'medium')]
students_medium['transport'].value_counts()

public       43
car owner    18
on foot       8
Name: transport, dtype: int64

In [27]:
(students_medium[students_medium['transport'] == 'on foot'].count() / students_medium.count())['userID']

0.11267605633802817

11% of students with a medium budget prefer walking to the restaurants.

---
## Questions - Part 2

**Question 8** What are the names of restaurants that do not have public parking lots ?

This question is ambiguous. Is 'street' a public parking ? Is a 'fee' parking public ? In the answer, I only retrieve restaurants whose parkings are not 'public'.

In [28]:
parking_names = restaurant_parking.merge(geoplaces, on='placeID')[['placeID', 'parking_lot', 'name']]
parking_names[parking_names['parking_lot'] != 'public'] [['name', 'parking_lot']]

,name,parking_lot
0,paniroles,NaN
1,potzocalli,NaN
2,el rincón de san francisco,NaN
3,vips,yes
5,mcdonalds parque tangamanga,yes
7,la estrella de dimas,NaN
8,el club,NaN
9,los toneles,NaN
10,koye sushi,NaN
11,restaurante pueblo bonito,yes


**Question 9** What are the addresses of restaurants which only accept cash ?

In [29]:
payment_names = accepts.merge(geoplaces, on='placeID')[['placeID', 'Rpayment', 'address']]
cash_restaurants = payment_names[payment_names['Rpayment'] == 'cash']
other_restaurants = payment_names[payment_names['Rpayment'] != 'cash']
only_cash_restaurants = cash_restaurants[~cash_restaurants['placeID'].isin(other_restaurants['placeID'])].dropna(axis='index')
only_cash_restaurants

,placeID,Rpayment,address
7,135088,cash,interior internado palmira sn
11,135085,cash,venustiano carranza 719 centro
12,135082,cash,villa de pozos 192 villa de pozos
26,135075,cash,himno nacional 2104 tangamanga
41,135070,cash,villa de pozos 4497 villa de pozos
42,135069,cash,industrias 908 valle dorado
53,135063,cash,alhondiga 62 centro
54,135062,cash,julian de los reyes 400 centro
55,135060,cash,ignacio allende 785 centro
56,135059,cash,santos degollado 745 los alamitos


In this question, we have to take care of restaurants accepting cash but also other forms of payment. This is why we remove this kind of restaurant in the second to last instruction.

**Question 10** Name the cities where the restaurants cook and serve ‘fast food’.

In [30]:
cuisine_cities = restaurant_cuisine.merge(geoplaces, on='placeID')[['placeID', 'city', 'Rcuisine']].dropna(axis='index')
cuisine_cities[cuisine_cities['Rcuisine'] == 'fast_food']['city'].unique()

array(['san luis potosi', 'soledad', 'cuernavaca', 'victoria'],
      dtype=object)

The cities where customers can find fast food are San Luis Potosi, Soledad, Cuernavaca and Victoria.

**Question 11** What is the most common 'rating' among customers with family ?

This question is ambiguous. Does having a family means being married, having kids, both ? In this answer, I assume it means 'having kids'.

In [38]:
family_rating = user_rating.merge(user_profile, on='userID')[['rating', 'hijos']]
family_rating[family_rating['hijos'] == 'kids']['rating'].value_counts()

0    39
2    28
1    15
Name: rating, dtype: int64

The most common rating for customers with kids is 0, apparently children prevent people from enjoying their dinner !

**Question 12** What types of 'rcuisine' received the highest rank from people with 'low'
budget ?

**Question 13** What is the average of 'service rating', received from 'social drinkers'
about restaurants which just served 'Wine-Beer' ?

**Question 14** How many smokers gave zero 'service rating' to the restaurants without an
open area ?

**Question 15** What is the average of 'service rating', received from 'social drinkers'
about restaurants which just served 'Wine-Beer' ?

**Question 16** Find the correlation between different rating categories
((general) rating, food_rating, service_rating) with the price levels of the
restaurants.